<a href="https://colab.research.google.com/github/Jw382716/ARIMA/blob/main/sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import os
current_path = os.getcwd()
print(current_path)
# 구글 코랩 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

/content
Mounted at /content/drive


In [ ]:
import zipfile

DATA_IN_PATH = '/content/drive/MyDrive/DeepLearning_NLP/data_in/'
file_list = ['labeledTrainData.tsv.zip', 'unlabeledTrainData.tsv.zip', 'testData.tsv.zip']

for file in file_list :
  zipRef = zipfile.ZipFile(DATA_IN_PATH + file, 'r')
  zipRef.extractall(DATA_IN_PATH)
  zipRef.close()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("파일크기 : ")
for file in os.listdir(DATA_IN_PATH) :
  if 'tsv' in file and 'zip' not in file :
    print(file.ljust(30) + str(round(os.path.getsize(DATA_IN_PATH + file) / 1000000,2)) + 'MB')
  
train_data = pd.read_csv(DATA_IN_PATH + 'labeledTrainData.tsv', header = 0, delimiter = '\t', quoting = 3)
train_data.head()

print('전체 학습데이터의 개수 : {}'.format(len(train_data)))

train_length = train_data['review'].apply(len)
train_length.head()

파일크기 : 
unlabeledTrainData.tsv        67.28MB
testData.tsv                  32.72MB
labeledTrainData.tsv          33.56MB
전체 학습데이터의 개수 : 25000


0    2304
1     948
2    2451
3    2247
4    2233
Name: review, dtype: int64

In [ ]:
import re 
import json 
import pandas as pd 
import numpy as np 
import nltk 
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords 
from tensorflow.keras.preprocessing.sequence import pad_sequences    
from tensorflow.keras.preprocessing.text import Tokenizer

DATA_IN_PATH = '/content/drive/MyDrive/DeepLearning_NLP/data_in/'
train_data = pd.read_csv(DATA_IN_PATH + 'labeledTrainData.tsv', header = 0, delimiter = '\t', quoting = 3)

review = train_data['review'][0]
review_text = BeautifulSoup(review, "html5lib").get_text()
review_text = re.sub('[^a-zA-Z]', ' ', review_text)
review_text = review_text.lower()
words = review_text.split()
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
print(words)
clean_review = ' '.join(words)
print(clean_review)

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocessing(review, remove_stopwords = False) :
  review_text = BeautifulSoup(review, "html5lib").get_text()
  review_text = re.sub('[^a-zA-Z]', ' ', review_text)
  review_text = review_text.lower()
  if remove_stopwords :
    words = review_text.split()
    stops = set(stopwords.words('english'))
    words = [w for w in words if not w in stops]
    review_text = ' '.join(words)
  return review_text

clean_train_reviews = []
for review in train_data['review'] :
  clean_train_reviews.append(preprocessing(review, remove_stopwords = True))
clean_train_df = pd.DataFrame({'review' : clean_train_reviews,
                               'sentiment' : train_data['sentiment']})

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)

MAX_SEQUENCE_LENGTH = 174
train_inputs = pad_sequences(text_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding = 'post')
print('Train Data :', train_inputs.shape)

train_labels = np.array(train_data['sentiment'])
print('Label:',train_labels.shape)

Train Data : (25000, 174)
Label: (25000,)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer = 'char', sublinear_tf=True, ngram_range=(1,3), max_features=5000)
x = vectorizer.fit_transform(reviews)
y = np.array(sentiments)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
lgs = LogisticRegression(class_weight = 'balanced')
lgs.fit(x_train, y_train)
print('Accuracy : %f'% lgs.score(x_test, y_test))

(20000, 5000) (5000, 5000) (20000,) (5000,)
Accuracy : 0.869600


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
x_train1 = x_train.toarray()
x_test1 = x_test.toarray()

model = keras.models.Sequential()
model.add(Dense(5000, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = "accuracy")
model.summary()


ValueError: ignored